<a href="https://colab.research.google.com/github/giuanm/inference_alerts_YOLO_FT/blob/main/hackathon_inference_alerts_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook para inferência do modelo YOLOv8 treinado,
processamento de vídeos e envio de alertas por e-mail.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print("Google Drive Montado.")

Mounted at /content/drive
Google Drive Montado.


In [ ]:
!pip install ultralytics==8.3.120 opencv-python-headless -q
print("Bibliotecas instaladas.")
!pip show ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.7 MB/s eta 0:00:00
Bibliotecas instaladas.
Name: ultralytics
Version: 8.3.120
Summary: Ultralytics YOLO 🚀 for SOTA object detection, multi-objec

In [ ]:
import cv2
import os
import time
import smtplib
from email.message import EmailMessage
from ultralytics import YOLO
from google.colab import userdata
import numpy as np

print("Bibliotecas importadas.")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Bibliotecas importadas.


In [ ]:
MODEL_PATH = '/content/drive/MyDrive/PosTech_IA_Devs_Hackathon/runs/detect/yolov8l_kc_v3_A100_final/weights/best.pt'

VIDEO_TEST_PATHS = [
    '/content/drive/MyDrive/PosTech_IA_Devs_Hackathon/video.mp4'
]

if not VIDEO_TEST_PATHS:
    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    print("!!! ATENÇÃO: Defina os caminhos corretos para os vídeos   !!!")
    print("!!! de teste na variável VIDEO_TEST_PATHS                !!!")
    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")


# Limiar de confiança para considerar uma detecção válida (0.0 a 1.0)
CONFIDENCE_THRESHOLD = 0.65

# Tempo de espera (em segundos) antes de enviar outro alerta para a MESMA classe
COOLDOWN_SECONDS = 3

print(f"Carregando modelo FINAL: {MODEL_PATH}")
if os.path.exists(MODEL_PATH):
    try:
        model = YOLO(MODEL_PATH)
        print("Modelo carregado com sucesso.")
        if hasattr(model, 'names') and isinstance(model.names, list):
             print("Classes do modelo carregado:", model.names)
             print(f"Número de classes (nc): {len(model.names)}")
        else:
             print("!!! ERRO INESPERADO: Modelo carregado não tem 'names' ou não é uma lista válida.")

    except Exception as e:
        print(f"ERRO ao carregar o arquivo do modelo em {MODEL_PATH}: {e}")
        print("O arquivo pode estar corrompido. Verifique o caminho.")
        raise

else:
    print(f"ERRO: Arquivo do modelo não encontrado em {MODEL_PATH}")
    print("Verifique se o caminho está correto no Google Drive e se o arquivo existe.")
    raise FileNotFoundError(f"Modelo não encontrado: {MODEL_PATH}")

last_alert_time = {}

PROJECT_DRIVE_PATH = '/content/drive/MyDrive/PosTech_IA_Devs_Hackathon'
LOG_FILE_PATH = os.path.join(PROJECT_DRIVE_PATH, 'detection_alerts_final_delivery.log')
print(f"Alertas serão salvos em: {LOG_FILE_PATH}")

Carregando modelo FINAL: /content/drive/MyDrive/PosTech_IA_Devs_Hackathon/runs/detect/yolov8l_kc_v3_A100_final/weights/best.pt
Modelo carregado com sucesso.
!!! ERRO INESPERADO: Modelo carregado não tem 'names' ou não é uma lista válida.
Alertas serão salvos em: /content/drive/MyDrive/PosTech_IA_Devs_Hackathon/detection_alerts_final_delivery.log


In [ ]:
# # Carrega as credenciais de e-mail dos Secrets do Colab
# try:
#     sender_email = userdata.get('SENDER_EMAIL')
#     email_password = userdata.get('EMAIL_PASSWORD')
#     receiver_email = userdata.get('RECEIVER_EMAIL')

#     if not sender_email or not email_password or not receiver_email:
#         raise ValueError("Uma ou mais credenciais de e-mail não foram encontradas nos Secrets.")
#     print("Credenciais de e-mail carregadas dos Secrets.")

# except Exception as e:
#     print(f"ERRO ao carregar credenciais dos Secrets: {e}")
#     print("Verifique se os Secrets 'SENDER_EMAIL', 'EMAIL_PASSWORD', 'RECEIVER_EMAIL' foram criados corretamente e habilitados.")


In [ ]:
PROJECT_DRIVE_PATH = '/content/drive/MyDrive/PosTech_IA_Devs_Hackathon'
LOG_FILE_PATH = os.path.join(PROJECT_DRIVE_PATH, 'detection_alerts.log')
print(f"Alertas serão salvos em: {LOG_FILE_PATH}")

def log_detection_alert(object_class, timestamp_str, confidence_pct):
    """Registra um alerta de detecção em um arquivo de log."""
    # Formata a mensagem de log com timestamp atual e detalhes da detecção
    log_timestamp = time.strftime('%Y-%m-%d %H:%M:%S')
    alert_message = f"[{log_timestamp}] ALERTA: {object_class} | Tempo Vídeo: {timestamp_str} | Confiança: {confidence_pct:.1f}%"

    print("*" * (len(alert_message) + 4))
    print(f"* {alert_message} *")
    print("*" * (len(alert_message) + 4))

    try:
        with open(LOG_FILE_PATH, 'a+') as log_file:
            log_file.write(alert_message + "\n")
        return True
    except Exception as e:
        print(f"\n!!! ERRO ao escrever no arquivo de log ({LOG_FILE_PATH}): {e} !!!\n")
        return False

if os.path.exists(LOG_FILE_PATH): os.remove(LOG_FILE_PATH)

print("\n--- Teste de Log ---")
test_logged = log_detection_alert("Foice de Teste", "00:15", 88.8)
print(f"Resultado do teste de log: {test_logged}")
print("--- Fim do Teste de Log ---\n")

Alertas serão salvos em: /content/drive/MyDrive/PosTech_IA_Devs_Hackathon/detection_alerts.log

--- Teste de Log ---
****************************************************************************************
* [2025-05-04 19:35:53] ALERTA: Foice de Teste | Tempo Vídeo: 00:15 | Confiança: 88.8% *
****************************************************************************************
Resultado do teste de log: True
--- Fim do Teste de Log ---



In [ ]:
print("\n--- Iniciando Processamento dos Vídeos ---")

PROCESSED_VIDEOS_DIR = os.path.join(PROJECT_DRIVE_PATH, 'processed_videos')
os.makedirs(PROCESSED_VIDEOS_DIR, exist_ok=True)
print(f"Vídeos processados serão salvos em: {PROCESSED_VIDEOS_DIR}")

for video_path in VIDEO_TEST_PATHS:
    print(f"\nProcessando vídeo: {video_path}")
    if not os.path.exists(video_path):
        print(f"ERRO: Vídeo não encontrado em {video_path}. Pulando.")
        continue

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"ERRO: Não foi possível abrir o vídeo: {video_path}. Verifique o caminho e o formato.")
        continue

    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps <= 0:
        print(f"Aviso: FPS do vídeo é inválido ({fps}). Usando 30 como padrão.")
        fps = 30.0
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print(f"Info do Vídeo: {frame_width}x{frame_height} @ {fps:.2f} FPS")

    output_filename = f"{os.path.splitext(os.path.basename(video_path))[0]}_processed_final_delivery.mp4"
    output_path = os.path.join(PROCESSED_VIDEOS_DIR, output_filename)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    if os.path.exists(output_path):
        print(f"Aviso: Arquivo de saída {output_path} já existe. Sobrescrevendo.")

    try:
        out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
        if not out.isOpened():
             print(f"!!! ERRO: Não foi possível inicializar cv2.VideoWriter para {output_path}. Verifique o codec ('mp4v'). !!!")
             cap.release()
             continue
        print(f"Configurado para salvar vídeo processado em: {output_path}")
    except Exception as e_writer_init:
        print(f"!!! ERRO Inesperado ao inicializar VideoWriter: {e_writer_init} !!!")
        cap.release()
        continue

    frame_count = 0
    processed_start_time = time.time()

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Fim da leitura do vídeo ou erro.")
            break

        frame_count += 1

        try:
            results = model(frame, verbose=False)
        except Exception as e_inference:
             print(f"!!! ERRO durante a inferência no frame {frame_count}: {e_inference} !!! Pulando inferência para este frame.")
             annotated_frame = frame.copy()
             results = None
             pass

        current_msec = cap.get(cv2.CAP_PROP_POS_MSEC)
        total_seconds = current_msec / 1000.0
        minutes = int(total_seconds // 60)
        seconds = int(total_seconds % 60)
        milliseconds = int((total_seconds - minutes * 60 - seconds) * 1000)
        timestamp_str = f"{minutes:02d}:{seconds:02d}:{milliseconds:03d}"

        annotated_frame = frame.copy()
        if results is not None and isinstance(results, list) and len(results) > 0:
            try:
                 annotated_frame = results[0].plot()
            except Exception as e_plot:
                 print(f"      !!! ERRO ao plotar frame {frame_count}: {e_plot} !!! Usando frame original.")

        try:
            out.write(annotated_frame)
        except Exception as e_write:
             print(f"      !!! ERRO ao escrever frame {frame_count}: {e_write} !!!")
             pass

        if results is not None and isinstance(results, list) and len(results) > 0 and hasattr(results[0], 'boxes') and results[0].boxes is not None:
            for box in results[0].boxes:
                try:
                    confidence = box.conf[0].item()
                    class_id = int(box.cls[0].item())

                    if confidence >= CONFIDENCE_THRESHOLD:
                        object_class = "Objeto Cortante Detectado"

                        try:
                            object_class_named = model.names[class_id]
                        except Exception:
                            object_class_named = f"Classe {class_id} (Nome Desconhecido)"


                        confidence_pct = confidence * 100

                        current_time = time.time()
                        last_time_alerted = last_alert_time.get(object_class_named, 0)

                        if (current_time - last_time_alerted) > COOLDOWN_SECONDS:
                            alert_processed = log_detection_alert(object_class_named, timestamp_str, confidence_pct)
                            if alert_processed:
                                last_alert_time[object_class_named] = current_time

                except Exception as e_alert_logic:
                    print(f"      !!! ERRO na lógica de alerta para uma box no frame {frame_count} ({timestamp_str}) !!!: {e_alert_logic}")
                    pass

        if frame_count % int(fps) == 0:
             elapsed_proc_time = time.time() - processed_start_time
             print(f"  Processado frame {frame_count}, Tempo no vídeo: {timestamp_str}, Tempo decorrido: {elapsed_proc_time:.1f}s")

    print(f"\nFinalizando processamento do vídeo {os.path.basename(video_path)}.")
    cap.release()
    if 'out' in locals() and out.isOpened():
        out.release()
        print(f"Gravador de vídeo liberado. Arquivo salvo em: {output_path}")
    elif 'out' in locals():
        print(f"Gravador de vídeo ('out') foi definido mas não estava aberto no final.")
    else:
        print("Gravador de vídeo ('out') não foi definido.")


    processed_end_time = time.time()
    print(f"Vídeo {os.path.basename(video_path)} processado e salvo em {output_path} em {processed_end_time - processed_start_time:.2f} segundos.")

print("\n--- Processamento de Todos os Vídeos Concluído ---")


--- Iniciando Processamento dos Vídeos ---
Vídeos processados serão salvos em: /content/drive/MyDrive/PosTech_IA_Devs_Hackathon/processed_videos

Processando vídeo: /content/drive/MyDrive/PosTech_IA_Devs_Hackathon/video.mp4
Info do Vídeo: 1920x1080 @ 30.00 FPS
Configurado para salvar vídeo processado em: /content/drive/MyDrive/PosTech_IA_Devs_Hackathon/processed_videos/video_processed_final_delivery.mp4
  Processado frame 30, Tempo no vídeo: 00:00:966, Tempo decorrido: 5.9s
  Processado frame 60, Tempo no vídeo: 00:01:966, Tempo decorrido: 7.1s
***********************************************************************************
* [2025-05-04 19:36:07] ALERTA: knife | Tempo Vídeo: 00:02:800 | Confiança: 65.4% *
***********************************************************************************
  Processado frame 90, Tempo no vídeo: 00:02:966, Tempo decorrido: 8.5s
  Processado frame 120, Tempo no vídeo: 00:03:966, Tempo decorrido: 10.1s
**************************************************